# TME sur les données blablacar

**Ce fichier est le fichier de travail**, l'autre fichier blablacar est donné pour information et pour montrer comment les données ont été collectées.


Zeng Fanxiang - Fu Zhenyue


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pickle as pkl

## Chargement des données

Les données sont stockées au format pickle (code fourni ci-dessous):

1. Importer le module : import `pickle as pkl`
1. Charger les données avec `load`
1. La structure est un dictionnaire, les données sont dans le champ `data`
1. La description des colonnes est dans `indexcol`


In [ ]:
# chargement des données
fich = pkl.load(open("donnees_blablacar.pkl", "rb"))

# {'indexcol': cols , 'data':pp2db, 'villes': villes, 'marques':marques }
titles_col = fich["indexcol"]
print(len(titles_col), titles_col)
data = fich["data"]
print(data.shape)
dico_villes = fich["villes"]
dico_marques = fich["marques"]
print(dico_marques)


## Discrétisation et histogramme

Nous nous intéressons à la variable `distance` (dernière colonne). Nous allons procéder de la manière suivante:

1. Analyse rapide de la variable aléatoire: calcul de la moyenne et de l'écart-type
1. Analyse plus fine (1): affichage des 10 quantiles
1. Analyse plus fine (2): discrétisation de la variable en 10 intervalles de largeur constante & comptage des effectifs dans chaque catégorie (= construction d'un histogramme)

- Construire l'histogramme à la main. (1) calculer les bornes des intervalles puis (2) utiliser `np.where` pour déterminer les effectifs dans chaque classe. Utiliser `plt.bar` pour l'affichage.
- Vérifier vos résultats avec `np.histogram` et `plt.hist` pour l'affichage
- Comparer les quantiles et les bornes des intervalles discrets

1. Discuter le nombre d'intervalles pour l'histogramme et trouver une valeur satisfaisante

**Notes** :

- dans `np.where`, il faut mettre des parenthèses s'il y a plusieurs clause

> `np.where((x>a) & (x<b))` : tous les indices de x qui satisfont la clause
>
> `np.where((x>a) & (x<b), 1, 0).sum()` : le comptage associé

- Dans `plt.bar`, il faut donner une largeur importante aux bar, sinon on ne voit rien


In [ ]:
# Analyse rapide : moyenne, écart-type, calcul des quantiles pour faire la synthèse de cette variable aléatoire

d = data[:, -1]  # extraction de la variable distance
print(d.shape)
print(d)
moy = np.mean(d)
ecart_type = np.std(d)
print("moyenne = ", moy, "km")
print("écart-type = ", ecart_type, "km")
quant = np.quantile(d, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
print("quantiles = ", quant)
hist = []
min = np.min(d)
max = np.max(d)
a = (max - min) / 10
print("distance", a)
b = min
for i in range(10):
    print(b, b + a)
    hist.append(np.where((d >= b) & (d <= (b + a)), 1, 0).sum())
    b += a

print(hist)


# A vous de jouer pour calculer tous les descripteurs = recherche dans la doc numpy !


In [ ]:
# Discrétisation des distances & histogramme avec np.histogram
counts, bins = np.histogram(d, bins=10)
print(counts)

plt.hist(d, bins=10)
plt.show()


In [ ]:
# Discrétisation des distances & histogramme a la main
n = 10  # nb intervalle
inter = np.linspace(np.min(d), np.max(d), n + 1)

print(inter)

bornes = np.zeros(n)
# calcul des effectifs avec np.where
for i in range(n):
    effectifs = np.where((d >= inter[i]) & (d <= (inter[i + 1])), 1, 0).sum()
    bornes[i] = effectifs
    # print(effectifs)

# affichage avec plt.bar = histogramme à la main
plt.bar(np.arange(n), bornes)
plt.show()


## histogramme (bis)

Tracer l'histogramme des prix au km


In [ ]:
# histogramme des prix au km : construction de la variable puis utilisation de np.histogramme
prix = data[:, -4]
# print(prix)
counts, bins = np.histogram(prix, bins=10)
plt.hist(prix, bins=10)
plt.show()


# Distributions jointes, distributions conditionnelles

Nous voulons maintenant étudier la distribution jointe entre la distance et la marque de la voiture. Partir des distributions discrètes ou discétisées et construire le tableau d'effectif puis normaliser par les effectifs de l'échantillon pour estimer la loi jointe.

Il est diffile d'analyser cette probabilité jointe (cf ci-dessous pour l'affichage)... Nous allons donc passer à la loi conditionnelle: nous voulons donc calculer la probabilité de la distance conditionnellement à la marque de la voiture.

1. Proposer un critère rapide pour vérifier que votre distribution conditionnelle respecte bien les propriétés de base
1. Cette distribution conditionnelle fait apparaitre des pics très marqués: pouvons-nous tirer parti de ces informations?

**Note:**

- pour afficher une matrice `p_dm`, la meilleure solution est la suivante:
  > `plt.imshow(p_dm, interpolation='nearest')`
  >
  > `plt.show()`
- la variable `marque` est bruitée. Vous pourrez vous amuser à éliminer ou fusionner certaines catégories
- les indices dans une matrice doivent toujours être entiers. `int(...)`
- pour ajouter une description sur l'axe des x:

```python
fig, ax = plt.subplots(1,1)
plt.imshow(p_dsm, interpolation='nearest')
ax.set_xticks(np.arange(len(dico_marques)))
ax.set_xticklabels(dico_marques.keys(),rotation=90,fontsize=8)
plt.show()
```

- Si l'image est trop petite pour voir quelque chose: solution = sauvegarde en pdf (ie vectorielle) + ouverture avec un logiciel de lecture pdf

```python
plt.savefig('mafigure.pdf')
```


In [ ]:
# print(dico_marques)
marque = data[:, -3]

marque_d = np.vstack((marque, d))
print(marque_d.shape)
print(marque_d)
norm = np.linalg.norm(marque_d, axis=0)
print(norm)
marque_d = marque_d / norm
print(marque_d)
print(marque_d.shape)
plt.imshow(marque_d, cmap="hot", interpolation="nearest")
plt.show()


In [ ]:
# loi jointe distance / marque
marque = data[:, -3]
# construction de la distance discrétisée
# Dimensions : = (Nind x 1) = mêmes dimensions que d
# contenu = catégorie de distance (entre 0 et 29 par exemple si on a discrétisé en 30 catégories)
dd = np.zeros(d.shape)
n = 30
inter = np.linspace(np.min(d), np.max(d), n + 1)

# print(inter)
for i in range(n):
    dd = np.where((d >= inter[i]) & (d <= (inter[i + 1])), i, dd)
print(dd)

# remplissage avec np.where

p_dm = np.zeros((len(inter) - 1, len(dico_marques)))

# remplissage de la matrice p_dm = double boucle + comptage
for i in range(p_dm.shape[0]):
    for j in range(p_dm.shape[1]):
        p_dm[i, j] = np.where((dd == i) & (marque == j), 1, 0).sum()

p_dm /= p_dm.sum()  # normalisation

print(p_dm)

# affichage du résultat
plt.imshow(p_dm, interpolation="nearest")
plt.show()


In [ ]:
# loi conditionnelle distance | marque
# calcul d'une marginale

p_m = np.zeros(len(dico_marques))
for i in range(p_m.shape[0]):
    p_m[i] = np.where(marque == i, 1, 0).sum()
pm = p_m / p_m.sum()  # normalisation

print(p_m.shape)
print(p_dm.shape)

# calcul de la conditionnelle
p_dsm = p_dm / p_m

# affichage
fig, ax = plt.subplots(1, 1)
plt.imshow(p_dsm, interpolation="nearest")
ax.set_xticks(np.arange(len(dico_marques)))
ax.set_xticklabels(dico_marques.keys(), rotation=90, fontsize=8)
plt.show()

# proposition d'un critère très rapide pour vérifier qu'il s'agit bien d'une distribution conditionnelle


## Tracé de l'ensemble de l'échantillon avec des codes couleurs

Nous proposons ensuite de tracer toutes les trajectoires des voitures blablacar. Pour cela, il faut utiliser la commande `plt.plot`.
Vous devez optenir des étoiles à partir des 7 villes requêtes: `['Paris', 'Marseille', 'Grenoble', 'Lille', 'Strasbourg', 'Nantes', 'Bordeaux']`.
Mais on ne voit pas grand chose... Et ça prend beaucoup de temps à tracer avec une boucle for. On propose donc une série d'exercice pour mieux comprendre ce qui se passe.

1. Attention à l'ordre des arguments dans le plot:
   `plt.plot(tous_les_x, tous_les_y)`
   Afin de tracer des trajectoires, il faut envoyer les x et les y 2 par 2 dans une boucle `for`
1. Pour éviter les boucles, il existe une méthode `quiver` dédiée au tracé de champs de vecteurs: ça ira beaucoup plus vite qu'avec plot. Il faut juste bien comprendre les mécanismes d'échelles. Pour utiliser l'échelle 1, la commande est la suivante:

```python
plt.quiver(x_dep, y_dep, delta_x, delta_y,\
            angles='xy', scale_units='xy', scale=1)
```

1. Isoler les trajets proposés à partir de chacune des villes sachant les coordonnées sont:

```python
coord = np.array([[45.18721767,  5.72345183],
 [47.22572172, -1.56558993],
 [50.63010695,  3.07071992],
 [48.5782548,   7.74078742],
 [44.83848889, -0.58156509],
 [43.2991509,   5.38925024],
 [48.8477201,   2.34607889]])
```

Chaque trajectoire (point de départ) sera rattachée à la ville la plus proche.
Une fois la distance calculée pour chaque origine de trajectoire, vous pourrez avoir besoin de `argmin`

1. Tracer les trajets d'une couleur spéciale en fonction des origines.

Les commandes matplotlib attendent des instructions de couleur au format RGB ou avec des lettres. Je vous propose une solution élégante pour distinguer les villes.

- soit l'index de la ville la plus proche sotcké dans `ville_or` (0,...,7)
- construire le dictionnaire: `dict({0:'b', 1:'r', 2:'k', 3:'y', 4:'c', 5:'m', 6:'g'})`
- transformer `ville_or` en `ville_c` en vectorisant l'appel à la table de hash:

```python
ville_c = np.vectorize(dico.get)(ville_or)
```


In [ ]:
# tracé de l'ensemble des trajectoires avec un code couleur en fonction de la ville de départ
ville = data[:, -2]
coord = data[:, 4:10]
print(
    [
        dico_villes[i]
        for i in [
            "Paris",
            "Marseille",
            "Grenoble",
            "Lille",
            "Strasbourg",
            "Nantes",
            "Bordeaux",
        ]
    ]
)
# print(coord)
arrive = [
    dico_villes[i]
    for i in [
        "Paris",
        "Marseille",
        "Grenoble",
        "Lille",
        "Strasbourg",
        "Nantes",
        "Bordeaux",
    ]
]
coord = coord[np.isin(coord[:, 1], arrive)]
print(coord_arr)
plt.plot(coord[:, 4], coord[:, 5], ".", color="red")  # coodonne arrivee
plt.plot(coord[:, 2], coord[:, 3], ".", color="blue")  # coodonne depart

plt.quiver(
    coord[:, 2],
    coord[:, 3],
    coord[:, 4] - coord[:, 2],
    coord[:, 5] - coord[:, 3],
    color="pink",
    angles="xy",
    scale_units="xy",
    scale=1,
)  # coodonne depart et arrivee
plt.show()


In [ ]:
# trouver l'information sur la ville la plus proche

coord = np.array([[45.18721767, 5.72345183],
                  [47.22572172, -1.56558993],
                  [50.63010695, 3.07071992],
                  [48.5782548, 7.74078742],
                  [44.83848889, -0.58156509],
                  [43.2991509, 5.38925024],
                  [48.8477201, 2.34607889]])

# calcul de la matrice de distance
dist =
# indice de la ville d'origine du trajet (plus petite distance dans le tableau ci-dessus)
ville_or =

# astuce pour construire une correspondance indice => code couleur
dico = dict({0: 'b', 1: 'r', 2: 'k', 3: 'y', 4: 'c', 5: 'm', 6: 'g'})
ville_c = np.vectorize(dico.get)(ville_or)

plt.figure()
plt.quiver([... coordonnées des vecteurs...]
color = ville_c, angles = 'xy', scale_units = 'xy', scale = 1)
plt.show()

## Etude de la corrélation entre variables

On propose d'étudier la corrélation entre la distance du trajet et le nombre d'étoiles de confort. Attention, les étoiles ne sont pas toujours renseignées (-1 = inconnu). On fera aussi ces opérations entre la distance et le prix.

1. Tracer dans le plan les coordonnées (distance,etoile) pour les points concernés

Vous utiliserez la commande `scatter` pour réaliser l'opération

1. Calculer le coefficient de corrélation entre les deux variables aléatoires


In [ ]:
# test de corrélation entre la distance et le confort de la voiture
plt.figure()
# affichage du nuage de points correspondant au deux variables
plt.show()

# calcul du coefficient de corrélation pour tous les points admissibles
# (ceux pour lesquels les étoiles sont renseignées)

# test de corrélation entre la distance et le prix
plt.figure()
# affichage du nuage de points correspondant au deux variables
plt.show()

# calcul du coefficient de corrélation pour tous les points admissibles


## Quelques questions supplémentaires

### prix au kilomètre en fonction de l'origine

On s'intérroge sur le prix des courses en fonction des villes de départ. On ne veut pas tomber dans des pièges liés à des résumés simplistes, nous allons donc calculer la distribution jointe (prix_km, ville_origine).

1. Mettre au propre le code de discretisation en construisant une méthode de signature:

```python
def discretisation(x, nintervalles):
    # [...]
    return bornes, effectifs, xd
```

1. Mettre au propre le code de calcul de la loi jointe entre deux échantillons de variables aléatoires discrètes

```python
def pjointe(xd, yd):
    # [...]
    return pj
```

1. En l'état, nous avons du mal à analyser les données. Ceci est du aux valeurs extrêmes (notamment hautes). Afin de rendre l'analyse robuste, seuiller le prix au km au 99ème percentile (toutes les valeurs supérieures sont ramenées à cette valeur limite).
1. Proposer quelques analyses.

### même analyse pour voir dans quelle ville les gens sont plus matinaux, s'ils partent plus vers le sud ou le nord, l'est ou l'ouest...

Si vous étiez un journaliste en manque de sujet de reportage, quel(s) graphique(s) calculeriez vous à partir de ces données?


In [ ]:
def discretisation(x, nintervalles, eps=0.0000001):
    # [...]
    return bornes, effectifs, xd


def pjointe(xd, yd):  # variable codées de 0 à valmax
    # [...]
    return pj


In [ ]:
# prix km
pkm = data[:, -4] / data[:, -1]
n = 30  # nb catégories

# robustesse : calcul du 99è percentile et seuillage des valeurs

bornes, effectifs, pkmd = discretisation(pkm, n)

pj = pjointe(pkmd, ville_or)

pv =  # calcul de la marginale
p_km_s_v =  # calcul de la conditionnelle

fig, ax = plt.subplots(1, 1)
plt.imshow(p_km_s_v, interpolation='nearest')
plt.show()

# analyses